In [1]:
def parse_input_part_two(file_path):
    """
    Parses the input file and extracts the seed ranges and maps.
    Expands the ranges into actual seed numbers.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    seeds = []
    maps = []
    current_map = []

    for line in lines:
        line = line.strip()
        if line.startswith("seeds:"):
            # Extract and expand seed ranges
            seed_pairs = list(map(int, line.replace("seeds:", "").strip().split()))
            for i in range(0, len(seed_pairs), 2):
                start, length = seed_pairs[i], seed_pairs[i + 1]
                seeds.extend(range(start, start + length))
        elif "map:" in line:
            # Start of a new map
            if current_map:
                maps.append(current_map)
                current_map = []
        elif line:
            # Line contains map data
            current_map.append(list(map(int, line.split())))

    if current_map:
        maps.append(current_map)

    return seeds, maps


def process_mapping(value, mapping):
    """
    Processes a single value through a map.
    If the value falls within a source range in the map, it is mapped to the corresponding
    destination range. If not, it maps to itself.
    """
    for dest_start, source_start, length in mapping:
        if source_start <= value < source_start + length:
            offset = value - source_start
            return dest_start + offset
    return value


def process_all_mappings(seeds, maps):
    """
    Processes each seed through all maps to determine the corresponding location numbers.
    Returns a list of location numbers.
    """
    locations = []
    for seed in seeds:
        value = seed
        for mapping in maps:
            value = process_mapping(value, mapping)
        locations.append(value)
    return locations


# Parse the input
file_path = 'sample-input.txt'  # Replace with the actual path to your input file
seeds, maps = parse_input_part_two(file_path)

# Process the seeds through all mappings
locations = process_all_mappings(seeds, maps)

# Find the lowest location number
lowest_location = min(locations)
print(f"The lowest location number is: {lowest_location}")

The lowest location number is: 46


In [ ]:
from heapq import heappush, heappop


def preprocess_map(mapping):
    """
    Converts a map into a dictionary for faster lookups.
    """
    map_dict = {}
    for dest_start, source_start, length in mapping:
        for i in range(length):
            source = source_start + i
            dest = dest_start + i
            map_dict[source] = dest
    return map_dict


def parse_input_optimized(file_path):
    """
    Parses the input file, expands seed ranges, and preprocesses maps into dictionaries.
    """
    with open(file_path, 'r') as file:
        lines = file.readlines()

    seeds = []
    maps = []
    current_map = []

    for line in lines:
        line = line.strip()
        if line.startswith("seeds:"):
            # Expand seed ranges
            seed_pairs = list(map(int, line.replace("seeds:", "").strip().split()))
            seeds = [
                seed
                for i in range(0, len(seed_pairs), 2)
                for seed in range(seed_pairs[i], seed_pairs[i] + seed_pairs[i + 1])
            ]
        elif "map:" in line:
            if current_map:
                maps.append(preprocess_map(current_map))
                current_map = []
        elif line:
            current_map.append(list(map(int, line.split())))

    if current_map:
        maps.append(preprocess_map(current_map))

    return seeds, maps


def process_all_mappings_optimized(seeds, maps):
    """
    Processes all seeds through preprocessed maps, finding the minimum location.
    """
    min_heap = []
    for seed in seeds:
        value = seed
        for map_dict in maps:
            value = map_dict.get(value, value)  # Default to itself if not in map
        heappush(min_heap, value)  # Push to heap for efficient minimum tracking
    return heappop(min_heap)  # Return the smallest location


# Parse the input and preprocess maps
file_path = 'input.txt'  # Replace with your actual input file path
seeds, maps = parse_input_optimized(file_path)

# Process seeds and find the lowest location
lowest_location = process_all_mappings_optimized(seeds, maps)
print(f"The optimized lowest location number is: {lowest_location}")